### Adding Tensorboard to the mix

In this notebook, we will add support for Tensorboard to allow us to track our results across multiple tests.  We we said in [Model Definition 2](https://github.com/adamx97/Data-Science-Advanced-Capstone/blob/master/Model%20Definition2--Advanced%20Data%20Science%20Capstone.ipynb)

> "We need to track the metrics of various models and parameter combinations, with the ability to backtrack to a former configuration if we find we are going down the wrong path.  We will use Tensorboard for this."

We want to be able to easily add models and track their evolution and performance in Tensorboard.  We will start with the same models we used in the Model Definistion 2 notebook, but add Tensorboard support.


## Initial Model Exploration 
Our research into initial models for Text Classification used two models shown below:

### Phase 1: First test model: Embedding, 1D Convolutional model with GlobalMaxPooling

This model was based on a sample Text Classification task that we found.  We vectorized our data with a simple integer encoding.  It uses embedding to turn our input vectors into dense vectors of fixed size.  Word embeddings encode each word into a dense vector that learns about its relative meaning within the training text.  By sliding a convolutional window over the vectors, there is the increased possibility that training will learn the meaning of various word combinations more easily than single or double word (our ngrams) occurances.  Our activation layer uses sigmoid to refine our results to a binary classification.

x = layers.Embedding(max_features + 1, embedding_dim)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)


The results of our iniitial tests with this model are shown below.

### Phase 2: Second test model: 
This model was based on a second sample Text Classification task that we found.  We vectorized our data using tf-idf (term-frequency times inverse document-frequency), a common weighting scheme used for document classification (see Feature Engineering notebook for more).  It uses fully 3 connected dense layers alternating with Dropout layers. We experimented with tanh and relu activations in this notebook.  We will further test varying the dropout rate, possibly testing it at each layer. As above, our activation layer uses sigmoid to refine our results to a binary classification.

x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='tanh')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

The results of our iniitial tests with this model are shown below.



In [ ]:
!pip install --upgrade numpy
!pip install --upgrade pandas

# we want tensorflow 2.3
!pip install --upgrade tensorflow  

In [1]:
import tensorflow as tf
print("Tensorflow version: ", tf.__version__)
if not tf.__version__ == '2.3.0':
    raise ValueError('please upgrade to TensorFlow 2.3, or restart your Kernel (Kernel->Restart & Clear Output)')
from tensorflow import keras

# Load the TensorBoard notebook extension.
%load_ext tensorboard

Tensorflow version:  2.3.0


In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from time import time
import logging
import numpy as np
import pandas as pd
import string
import re
from datetime import datetime
from packaging import version

from keras.utils import to_categorical
from keras import models
from keras import layers

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from sklearn.model_selection import train_test_split

from ibm_botocore.client import Config
import ibm_boto3

Using TensorFlow backend.


In [3]:
#Get our data
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_news = {
    'IAM_SERVICE_ID': 'iam-ServiceId-32e8ee67-397c-4ff1-b69b-543172331f43',
    'IBM_API_KEY_ID': 'Rx4FR4JSAueCnnIsoevsgYgOsuh8LCXtbkFpFpC0EmVU',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'advanceddatasciencecapstone-donotdelete-pr-tqabpnbxebk8rm',
    'FILE': 'dfTrueFalseNews.pkl'
}

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')
        
dfNews = download_file_cos(credentials_news, "dfTrueFalseNews.pkl", "dfTrueFalseNews.pkl")

File Downloaded


In [4]:
dfNews = pd.read_pickle('dfTrueFalseNews.pkl')
#dfNews['truthvalue'] = pd.Categorical(dfNews['truthvalue'])

print (dfNews.shape, dfNews.columns, '\n',  dfNews.dtypes)

(1126, 3) Index(['text', 'source', 'truthvalue'], dtype='object') 
 text          object
source        object
truthvalue    object
dtype: object


In [5]:
dfNews.head()

,text,source,truthvalue
tech003legit,A Google computer victorious over the world's ...,MihalceaNewsLegit,1
polit11legit,White House keeps up sanctuary cities pressure...,MihalceaNewsLegit,1
biz40legit,Why Silicon Valley isn't fighting to save the ...,MihalceaNewsLegit,1
edu10legit,Protesters Disrupt DeVos School Visit Protes...,MihalceaNewsLegit,1
tech038legit,Solar-powered 'skin' could make prosthetics mo...,MihalceaNewsLegit,1


In [6]:
x = dfNews['text'].values
y = dfNews['truthvalue'].values
print(type(x), type(y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [7]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

# Once we have our handles, we format the datasets in a Keras-fit compatible
# format: a tuple of the form (text_data, label).
def format_dataset(x, y):
  return (x, y)

train_dataset = list(map(format_dataset, X_train, y_train))
test_dataset = list(map(format_dataset, X_test, y_test))

# We also create a dataset with only the textual data in it. This will be used
# to build our vocabulary later on.
textL_dataset = list(map(lambda a:a, x))


In [8]:
print (len(X_train), len(X_test), len(y_train), len(y_test), len(textL_dataset), '\n',
type(X_train), type(X_test), type(y_train), type(y_test), type(textL_dataset))


900 226 900 226 1126 
 <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'>


In [9]:
# move our numpy structures into Tensorflow datasets
Dataset = tf.data.Dataset
text_dataset = tf.data.Dataset.from_tensor_slices(textL_dataset)

features_train_dataset = Dataset.from_tensor_slices(X_train)
labels_train_dataset = Dataset.from_tensor_slices(list(y_train))
tfds_train = Dataset.zip((features_train_dataset, labels_train_dataset))

features_test_dataset = Dataset.from_tensor_slices(X_test)
labels_test_dataset = Dataset.from_tensor_slices(list(y_test))
tfds_test = Dataset.zip((features_test_dataset, labels_test_dataset))

## Phase 1: Adding Tensorboard to Model 1  basic vectorization and Embedding and Convolutional 1d model

In [10]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

# Model constants.
max_features = 20000 #vocabulary size
embedding_dim = 128

# We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer1 = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=500)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.
vectorize_layer1.adapt(text_dataset.batch(64))


# A text input.
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')

# The first layer in our model is the vectorization layer. After this layer,
# we have a tensor of shape (batch_size, max_len) containing vocab indices.
x = vectorize_layer1(text_input)

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'. Note that we're using max_features+1 here, since there's an
# OOV token that gets added to the vocabulary in vectorize_layer.
x = layers.Embedding(max_features + 1, embedding_dim)(x)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)

x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model1 = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model1.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
batch_size = 32
epochs = 1

# Tensorboard callback
logdir = "logs/scalars/model1_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Fit the model using the train and test datasets.
training_history = model1.fit(
    tfds_train.batch(batch_size),
    validation_data=tfds_test.batch(batch_size),
    epochs=epochs,
    callbacks=[tensorboard_callback])

print("Average test loss: ", np.average(training_history.history['loss']))

 1/29 [>.............................] - ETA: 0s - loss: 0.6936 - accuracy: 0.4062

W0901 20:28:01.829129 140568521832256 deprecation.py:323] From /opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


29/29 [==============================] - 8s 269ms/step - loss: 0.6965 - accuracy: 0.4711 - val_loss: 0.6954 - val_accuracy: 0.4823
Average test loss:  0.6964858174324036


In [12]:
# Model 1 using 2 ngrams
vectorize_layer1 = TextVectorization(
    ngrams=2,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=500)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.
vectorize_layer1.adapt(text_dataset.batch(64))

batch_size = 64
epochs = 3

# Fit the model using the train and test datasets.
model1.fit(
    tfds_train.batch(batch_size),
    validation_data=tfds_test.batch(batch_size),
    epochs=epochs)

Epoch 1/3
15/15 [==============================] - 5s 346ms/step - loss: 0.6896 - accuracy: 0.5567 - val_loss: 0.6966 - val_accuracy: 0.4823
Epoch 2/3
15/15 [==============================] - 5s 340ms/step - loss: 0.6918 - accuracy: 0.5244 - val_loss: 0.6939 - val_accuracy: 0.4867
Epoch 3/3
15/15 [==============================] - 5s 323ms/step - loss: 0.6883 - accuracy: 0.5433 - val_loss: 0.6967 - val_accuracy: 0.4867


In [15]:
!pip install --upgrade grpcio

     |████████████████████████████████| 3.3MB 6.1MB/s eta 0:00:01
  Found existing installation: grpcio 1.16.1
    Uninstalling grpcio-1.16.1:
      Successfully uninstalled grpcio-1.16.1


In [17]:
%tensorboard --logdir logs/scalars
#S3_VERIFY_SSL=0 S3_ENDPOINT=s3-api.us-geo.objectstorage.softlayer.net AWS_ACCESS_KEY_ID=xxxxx AWS_SECRET_ACCESS_KEY=xxxxx AWS_REGION=us-geo tensorboard --logdir=s3://mnist-fashion-results/training--1u0V9RiR/learner-1/logs/tb > /dev/null

Reusing TensorBoard on port 6006 (pid 520), started 0:00:45 ago. (Use '!kill 520' to kill it.)

In [28]:
!cat logs/scalars/model1_20200901-202800/validation/events.out.tfevents.1598992088.notebook-a7d2ab4351974ce48182be28fae05a5d-78b569bc5b-5f7jq.172.1621.v2


brain.Event:2bı�       ��2	_�R����A*


epoch_loss�2?�&36"       x=�	��R����A*

epoch_accuracy$��>��T

### Phase 1: Model 1 Discussion
This is the first model we tested and the two runs shown above seemed to reach peak accuracy in the 2 to 3 epoch range, though we will test them further as we move forward.  Using 2 ngrams (vs. single words) in our TextVectorization layer improved accuracy about 1.8%.

## Model 2 TextVectorization layer in a bigram TF-IDF densely-connected model

In [ ]:

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers


# Model constants.
max_features = 14745 #vocabulary size


#We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer2 = TextVectorization(
    max_tokens=max_features,
    output_mode='tf-idf',
    ngrams=2,  # Unigrams and bigrams
    
    #output_sequence_length=500
    )

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.
vectorize_layer2.adapt(text_dataset.batch(64))



# A text input.
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')

# The first layer in our model is the vectorization layer. After this layer,
# we have a tensor of shape (batch_size, features) containing TF-IDF features.
x = vectorize_layer2(text_input)

# Dense hidden layers
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='tanh')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model2 = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model2.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs = 6

# Fit the model using the train and test datasets.
history = model2.fit(
    tfds_train.batch(batch_size),
    validation_data=tfds_test.batch(batch_size),
    epochs=epochs)

In [ ]:
#results = model2.evaluate(tfds_test)

#print(results)
history_dict = history.history
history_dict.keys()

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


Results with 4 layers, 3 epochs: 

Layers were: 
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)


Accuracy is improving, but validation accuracy is not improving,  seems like overfitting.
`  Epoch 1/3
  15/15 [==============================] - 39s 3s/step - loss: 1.1349 - accuracy: 0.5039 - val_loss: 0.6969 - val_accuracy: 0.5531
  Epoch 2/3
  15/15 [==============================] - 39s 3s/step - loss: 0.7954 - accuracy: 0.6246 - val_loss: 0.7491 - val_accuracy: 0.5310
  Epoch 3/3
  15/15 [==============================] - 39s 3s/step - loss: 0.5594 - accuracy: 0.7619 - val_loss: 0.7448 - val_accuracy: 0.5354
 `

Results with 6 layers, 3 epochs 128 units dense layer.
Layers were: 

x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

Both accuracy and val_accuracy are improving, seems better.
`
Epoch 1/3
15/15 [==============================] - 38s 3s/step - loss: 1.0789 - accuracy: 0.5028 - val_loss: 0.7001 - val_accuracy: 0.5265
Epoch 2/3
15/15 [==============================] - 37s 2s/step - loss: 0.9954 - accuracy: 0.5205 - val_loss: 0.6982 - val_accuracy: 0.5221
Epoch 3/3
15/15 [==============================] - 37s 2s/step - loss: 0.9662 - accuracy: 0.5692 - val_loss: 0.6842 - val_accuracy: 0.5708
`

Results with 6 layers, 3 epochs, 256 more units in the Dense layer. 18000 vocab

x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

`
Epoch 1/3
15/15 [==============================] - 35s 2s/step - loss: 1.3553 - accuracy: 0.4795 - val_loss: 0.7590 - val_accuracy: 0.4956
Epoch 2/3
15/15 [==============================] - 35s 2s/step - loss: 1.0434 - accuracy: 0.5703 - val_loss: 0.6703 - val_accuracy: 0.5575
Epoch 3/3
15/15 [==============================] - 35s 2s/step - loss: 0.8842 - accuracy: 0.6301 - val_loss: 0.6928 - val_accuracy: 0.5265
`

Results with 6 layers, 3 epochs, 256 units in Dense layer, and all single occurring words removed, leaving 14,745 vocabulary words.
`
Epoch 1/3
15/15 [==============================] - 29s 2s/step - loss: 1.2235 - accuracy: 0.4906 - val_loss: 0.8649 - val_accuracy: 0.4823
Epoch 2/3
15/15 [==============================] - 28s 2s/step - loss: 1.0063 - accuracy: 0.5847 - val_loss: 0.6744 - val_accuracy: 0.6062
Epoch 3/3
15/15 [==============================] - 28s 2s/step - loss: 0.7222 - accuracy: 0.6611 - val_loss: 0.6706 - val_accuracy: 0.5664
`

### Replace dense layer with tanh activation layer:

x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='tanh')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

Results with 6 layers, 6 epochs, 256 units in Dense layer, middle layer w/ tanh activation and all single occurring words removed, leaving 14,745 vocabulary words.
`
Epoch 1/6
15/15 [==============================] - 29s 2s/step - loss: 0.8455 - accuracy: 0.5083 - val_loss: 0.6888 - val_accuracy: 0.5221
Epoch 2/6
15/15 [==============================] - 28s 2s/step - loss: 0.7452 - accuracy: 0.5703 - val_loss: 0.7091 - val_accuracy: 0.5000
Epoch 3/6
15/15 [==============================] - 28s 2s/step - loss: 0.7064 - accuracy: 0.6213 - val_loss: 0.6763 - val_accuracy: 0.5796
Epoch 4/6
15/15 [==============================] - 28s 2s/step - loss: 0.5463 - accuracy: 0.7276 - val_loss: 0.7541 - val_accuracy: 0.5619
Epoch 5/6
15/15 [==============================] - 28s 2s/step - loss: 0.3879 - accuracy: 0.8295 - val_loss: 0.9690 - val_accuracy: 0.5841
Epoch 6/6
15/15 [==============================] - 28s 2s/step - loss: 0.2275 - accuracy: 0.9037 - val_loss: 1.1818 - val_accuracy: 0.5929
`

In [ ]:
history_dict = history.history
history_dict.keys()

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Results with 6 layers, 3 epochs, 256 more units in the Dense layer. 20000 vocab

x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
`
Epoch 1/3
15/15 [==============================] - 38s 3s/step - loss: 1.2857 - accuracy: 0.5316 - val_loss: 0.7208 - val_accuracy: 0.5398
Epoch 2/3
15/15 [==============================] - 38s 3s/step - loss: 1.1039 - accuracy: 0.5703 - val_loss: 0.6790 - val_accuracy: 0.5664
Epoch 3/3
15/15 [==============================] - 38s 3s/step - loss: 0.9360 - accuracy: 0.6523 - val_loss: 0.6857 - val_accuracy: 0.5885
`

In [ ]:
Results with 6 layers, 3 epochs, 256 units in dense layer, 25000 vocab.
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
`
Epoch 1/3
15/15 [==============================] - 49s 3s/step - loss: 1.6061 - accuracy: 0.5017 - val_loss: 0.6888 - val_accuracy: 0.5044
Epoch 2/3
15/15 [==============================] - 48s 3s/step - loss: 1.0667 - accuracy: 0.5415 - val_loss: 0.7125 - val_accuracy: 0.4912
Epoch 3/3
15/15 [==============================] - 48s 3s/step - loss: 1.0003 - accuracy: 0.6456 - val_loss: 0.6951 - val_accuracy: 0.5221
`

In [ ]:
model2.summary()

### Phase 2: Model 2 Discussion

This is the second model we tested. The primary difference from the first model is the textvectorization (tf-idf weighting vs. unweighted integer) and fully connected dense layers vs convolutional wones.   Here we extended our epochs to 6 and found that indeed, the 2nd or 3rd epoch tended to give the best results.  
 
We also varied activation layers (relu vs. tanh) and vocabulary size.  

Making multiple changes to our test in multiple dimensions made clear the challenges of maintaining clear notes concerning the model parameters and results obtained. the two runs shown above seemed to reach peak accuracy in the 2 to 3 epoch range, though we will test them further as we move forward. 

Thus we are motivated to put together a clearly structured lab environment for testing models, parameters, and storing results.  Luckily Tensorflow provides much of the framework for this.  This work is continued in the next notebook.


